In [11]:
#export https_proxy="http://localhost:8118"

install python api https://developers.google.com/api-client-library/python/start/installation
gmail message api :https://developers.google.com/gmail/api/v1/reference/users/messages/list


In [35]:
from __future__ import print_function
import httplib2
import os

from apiclient import discovery
import oauth2client
from oauth2client import client
from oauth2client import tools

service =None

try:
    import argparse
    #flags = argparse.ArgumentParser(parents=[tools.argparser]).parse_args()
except ImportError:
    flags = None

SCOPES = 'https://www.googleapis.com/auth/gmail.readonly'
CLIENT_SECRET_FILE = '/home/guci/shell/client_secret.json'
APPLICATION_NAME = 'Gmail API Python Quickstart'


def get_credentials():
    """Gets valid user credentials from storage.

    If nothing has been stored, or if the stored credentials are invalid,
    the OAuth2 flow is completed to obtain the new credentials.

    Returns:
        Credentials, the obtained credential.
    """
    home_dir = os.path.expanduser('~')
    credential_dir = os.path.join(home_dir, '.credentials')
    if not os.path.exists(credential_dir):
        os.makedirs(credential_dir)
    credential_path = os.path.join(credential_dir,
                                   'gmail-python-quickstart.json')

    store = oauth2client.file.Storage(credential_path)
    credentials = store.get()
    if not credentials or credentials.invalid:
        flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
        flow.user_agent = APPLICATION_NAME
        if flags:
            credentials = tools.run_flow(flow, store, flags)
        else: # Needed only for compatability with Python 2.6
            credentials = tools.run(flow, store)
        print('Storing credentials to ' + credential_path)
    return credentials

def initService():
    """Shows basic usage of the Gmail API.

    Creates a Gmail API service object and outputs a list of label names
    of the user's Gmail account.
    """
    credentials = get_credentials()
    http = credentials.authorize(httplib2.Http())
    global service
    service = discovery.build('gmail', 'v1', http=http)

    """
    results = service.users().labels().list(userId='me').execute()
    labels = results.get('labels', [])

    if not labels:
        print('No labels found.')
    else:
      print('Labels:')
      for label in labels:
        print(label['name'])
    """

In [64]:
"""Get a list of Messages from the user's mailbox.
"""

from apiclient import errors

def PrintMessage(mid):
    message = service.users().messages().get(userId='me', id=mid,
                                             format='raw').execute()
    print ('Message snippet: %s' % message['snippet'])

def ListMessagesMatchingQuery(service, user_id, query=''):
  """List all Messages of the user's mailbox matching the query.

  Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    query: String used to filter messages returned.
    Eg.- 'from:user@some_domain.com' for Messages from a particular sender.

  Returns:
    List of Messages that match the criteria of the query. Note that the
    returned list contains Message IDs, you must use get with the
    appropriate ID to get the details of a Message.
  """
  try:
    response = service.users().messages().list(userId=user_id,
                                               q=query).execute()
    messages = []
    if 'messages' in response:
      messages.extend(response['messages'])

    while 'nextPageToken' in response:
      page_token = response['nextPageToken']
      response = service.users().messages().list(userId=user_id, q=query,
                                         pageToken=page_token).execute()
      messages.extend(response['messages'])
      for message in messages:
            PrintMessage(message['id'])
      messages = []
    return messages
  except errors.HttpError, error:
    print ('An error occurred: %s' % error)


def ListMessagesWithLabels(service, user_id, label_ids=[]):
  """List all Messages of the user's mailbox with label_ids applied.

  Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    label_ids: Only return Messages with these labelIds applied.

  Returns:
    List of Messages that have all required Labels applied. Note that the
    returned list contains Message IDs, you must use get with the
    appropriate id to get the details of a Message.
  """
  try:
    response = service.users().messages().list(userId=user_id,
                                               labelIds=label_ids).execute()
    messages = []
    if 'messages' in response:
      messages.extend(response['messages'])

    while 'nextPageToken' in response:
      page_token = response['nextPageToken']
      response = service.users().messages().list(userId=user_id,
                                                 labelIds=label_ids,
                                                 pageToken=page_token).execute()
      messages.extend(response['messages'])

    return messages
  except errors.HttpError, error:
    print('An error occurred: %s' % error)

In [36]:
initService()

In [66]:
ListMessagesMatchingQuery(service,user_id='me',query='资源')

Message snippet: ﻿ 本日推荐文章 【网络民议】你胡侃是多元 别人胡侃进局子 纽约时报 | 王健林回应习近平亲属入股万达报道 海涛评论 | 当“天才徐翔”登上被收割者的名单 【习总日记】 贾庆林给十八届五中全会的贺信 端
Message snippet: , 您好! 原邮件信息 来信人: jpimd &lt;jpimd@mx.cei.gov.cn&gt; 日 期: Wed, 5 Aug 2015 17:12:25 +0800 主 题: 信息资源统一管理（
Message snippet: , 您好! 原邮件信息 来信人: liuft &lt;liuft@diasia.org.cn&gt; 日 期: Thu, 27 Aug 2015 21:56:38 +0800 主 题: 转发: Fw:
Message snippet: , 您好! 原邮件信息 来信人: 蒋平 &lt;jiangping@diasia.org.cn&gt; 日 期: Wed, 14 Oct 2015 11:41:08 +0800 主 题: 盛唐新世代发展
Message snippet: -------- 转发邮件信息 -------- 发件人：&quot;jpimd&quot; &lt;jpimd@mx.cei.gov.cn&gt; 发送日期：2015-08-25 16:56:32 收
Message snippet: ﻿ 本日推荐文章 羽戈：就放开二孩说两句 唐映红 ：如何看待人们对普遍放开“二胎”的不同反应？ 自由亚洲｜“十三五”纲要起草小组成立 贫困县五年脱贫受质疑 【习总日记】 温家宝给十八届五中全会的贺信
Message snippet: ﻿ 本日推荐文章 法广｜记者刘伟央视忏悔后获取保释放 【网络民议】推特都不能玩，我也要加入宣传部 【习总日记】王贼捅刀、胡贼补刀、习总负伤 【河蟹档案】70后的痛：你硬时政策比你硬，政策软时你还软 自由
Message snippet: 本日推荐文章 网易另一面 | 育儿压力不减 放开二孩难奏效 苹果日报 | 破例说明十三五规划建议 抢李克强工作 习近平显霸权 端传媒 | 谁在制造低价港澳游？起底陆港黑色旅游产业链 【网络民议】“一人
Message snippet: ﻿ 本日推荐文章 贾嘉：办个文学奖哄老婆开心 贵文化圈真是奇葩百出 【

[]